In [1]:
!pip install adversarial-robustness-toolbox

In [55]:
from __future__ import absolute_import, division, print_function, unicode_literals

import logging

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np

from art.attacks.evasion import DeepFool
from art.estimators.classification import KerasClassifier
from art.utils import load_dataset



In [56]:
# Configure a logger to capture ART outputs; these are printed in console and the level of detail is set to INFO
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter("[%(levelname)s] %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)

# Read CIFAR10 dataset
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str("cifar10"))
x_train, y_train = x_train[:100], y_train[:100]
x_test, y_test = x_test[:50], y_test[:50]
im_shape = x_train[0].shape


In [57]:

# Create Keras convolutional neural network - basic architecture from Keras examples
# Source here: https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py
model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", input_shape=x_train.shape[1:]))
# model.add(Activation("relu"))
# model.add(Conv2D(32, (3, 3)))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same"))
# model.add(Activation("relu"))
# model.add(Conv2D(64, (3, 3)))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation("relu"))
# model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])



In [58]:
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

In [59]:
import sys
print(sys.getrecursionlimit())
print(min_,max_)

10000
0.0 1.0


In [60]:
import sys
sys.setrecursionlimit(10000)

In [61]:
# Create classifier wrapper
classifier = KerasClassifier(model=model, clip_values=(0.0,1.0))
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)


[INFO] Inferred 6 hidden layers on Keras classifier.
[INFO] Inferred 6 hidden layers on Keras classifier.
[INFO] Inferred 6 hidden layers on Keras classifier.
[INFO] Inferred 6 hidden layers on Keras classifier.


Train on 100 samples
Epoch 1/10
100/100 [==============================] - 1s 7ms/sample - loss: 2.3456 - accuracy: 0.0700
Epoch 2/10
100/100 [==============================] - 0s 5ms/sample - loss: 4.9100 - accuracy: 0.1900
Epoch 3/10
100/100 [==============================] - 0s 5ms/sample - loss: 4.0360 - accuracy: 0.1300
Epoch 4/10
100/100 [==============================] - 0s 5ms/sample - loss: 3.0423 - accuracy: 0.2600
Epoch 5/10
100/100 [==============================] - 0s 5ms/sample - loss: 2.1526 - accuracy: 0.3300
Epoch 6/10
100/100 [==============================] - 0s 5ms/sample - loss: 2.4398 - accuracy: 0.2600
Epoch 7/10
100/100 [==============================] - 0s 5ms/sample - loss: 2.5563 - accuracy: 0.2600
Epoch 8/10
100/100 [==============================] - 0s 5ms/sample - loss: 1.7980 - accuracy: 0.4300
Epoch 9/10
100/100 [==============================] - 0s 5ms/sample - loss: 1.4528 - accuracy: 0.5800
Epoch 10/10
100/100 [==============================] - 0s 5ms

In [62]:
# Evaluate the original trained classifier on the test set
preds = np.argmax(classifier.predict(x_test), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
logger.info("Classifier with adversarial training")
logger.info("Accuracy on adversarial samples: %.2f%%", (acc * 100))

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
[INFO] Classifier with adversarial training
[INFO] Classifier with adversarial training
[INFO] Classifier with adversarial training
[INFO] Classifier with adversarial training
[INFO] Accuracy on adversarial samples: 20.00%
[INFO] Accuracy on adversarial samples: 20.00%
[INFO] Accuracy on adversarial samples: 20.00%
[INFO] Accuracy on adversarial samples: 20.00%


In [63]:
# Craft adversarial samples with DeepFool
logger.info("Create DeepFool attack")
adv_crafter = DeepFool(classifier)
logger.info("Craft attack on training examples")
x_train_adv = adv_crafter.generate(x_train)
logger.info("Craft attack test examples")
x_test_adv = adv_crafter.generate(x_test)

[INFO] Create DeepFool attack
[INFO] Create DeepFool attack
[INFO] Create DeepFool attack
[INFO] Create DeepFool attack
[INFO] Craft attack on training examples
[INFO] Craft attack on training examples
[INFO] Craft attack on training examples
[INFO] Craft attack on training examples
[WARNING] It seems that the attacked model is predicting probabilities. DeepFool expects logits as model output to achieve its full attack strength.
[WARNING] It seems that the attacked model is predicting probabilities. DeepFool expects logits as model output to achieve its full attack strength.
[WARNING] It seems that the attacked model is predicting probabilities. DeepFool expects logits as model output to achieve its full attack strength.
[WARNING] It seems that the attacked model is predicting probabilities. DeepFool expects logits as model output to achieve its full attack strength.


DeepFool:   0%|          | 0/100 [00:00<?, ?it/s]

[INFO] Craft attack test examples
[INFO] Craft attack test examples
[INFO] Craft attack test examples
[INFO] Craft attack test examples
[WARNING] It seems that the attacked model is predicting probabilities. DeepFool expects logits as model output to achieve its full attack strength.
[WARNING] It seems that the attacked model is predicting probabilities. DeepFool expects logits as model output to achieve its full attack strength.
[WARNING] It seems that the attacked model is predicting probabilities. DeepFool expects logits as model output to achieve its full attack strength.
[WARNING] It seems that the attacked model is predicting probabilities. DeepFool expects logits as model output to achieve its full attack strength.


DeepFool:   0%|          | 0/50 [00:00<?, ?it/s]

In [64]:
# Evaluate the classifier on the adversarial samples
preds = np.argmax(classifier.predict(x_test_adv), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
logger.info("Classifier before adversarial training")
logger.info("Accuracy on adversarial samples: %.2f%%", (acc * 100))



[INFO] Classifier before adversarial training
[INFO] Classifier before adversarial training
[INFO] Classifier before adversarial training
[INFO] Classifier before adversarial training
[INFO] Accuracy on adversarial samples: 20.00%
[INFO] Accuracy on adversarial samples: 20.00%
[INFO] Accuracy on adversarial samples: 20.00%
[INFO] Accuracy on adversarial samples: 20.00%


In [65]:
# Data augmentation: expand the training set with the adversarial samples
x_train = np.append(x_train, x_train_adv, axis=0)
y_train = np.append(y_train, y_train, axis=0)

In [66]:
# Retrain the CNN on the extended dataset
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)


Train on 200 samples
Epoch 1/10
200/200 [==============================] - 1s 6ms/sample - loss: 2.5149 - accuracy: 0.3350
Epoch 2/10
200/200 [==============================] - 1s 5ms/sample - loss: 1.6081 - accuracy: 0.4600
Epoch 3/10
200/200 [==============================] - 1s 5ms/sample - loss: 1.3967 - accuracy: 0.6300
Epoch 4/10
200/200 [==============================] - 1s 5ms/sample - loss: 0.9173 - accuracy: 0.7550
Epoch 5/10
200/200 [==============================] - 1s 5ms/sample - loss: 0.8770 - accuracy: 0.7000
Epoch 6/10
200/200 [==============================] - 1s 5ms/sample - loss: 0.6231 - accuracy: 0.8850
Epoch 7/10
200/200 [==============================] - 1s 5ms/sample - loss: 0.5000 - accuracy: 0.8950
Epoch 8/10
200/200 [==============================] - 1s 4ms/sample - loss: 0.4241 - accuracy: 0.9350
Epoch 9/10
200/200 [==============================] - 1s 4ms/sample - loss: 0.3447 - accuracy: 0.9650
Epoch 10/10
200/200 [==============================] - 1s 5ms

In [67]:
# Evaluate the adversarially trained classifier on the test set
preds = np.argmax(classifier.predict(x_test_adv), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
logger.info("Classifier with adversarial training")
logger.info("Accuracy on adversarial samples: %.2f%%", (acc * 100))

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
[INFO] Classifier with adversarial training
[INFO] Classifier with adversarial training
[INFO] Classifier with adversarial training
[INFO] Classifier with adversarial training
[INFO] Accuracy on adversarial samples: 14.00%
[INFO] Accuracy on adversarial samples: 14.00%
[INFO] Accuracy on adversarial samples: 14.00%
[INFO] Accuracy on adversarial samples: 14.00%
